# 용어와 매칭 함수

In [2]:
# !pip install PyMuPDF

In [3]:
import os
import re
import fitz          # PyMuPDF
import pandas as pd

def extract_text_from_pdf(pdf_path: str) -> str:
    doc = fitz.open(pdf_path)
    full_text = "".join(page.get_text() + "\n" for page in doc)
    doc.close()
    return full_text

def normalize_linebreaks(text: str) -> str:
    text = re.sub(r'(?<!\n)\n(?!\n)', '', text)
    text = re.sub(r'(?<=[\.!?])(?=[^\s])', ' ', text) # ? . ! 뒤에 공백이 없는 경우 의도적으로 붙여주기
    return text

def split_sentences(text: str, min_len: int = 10) -> list[str]:
    text = normalize_linebreaks(text)
    pattern = r'(?<=[\.!?])(?:\s+|\n)|(?<=[다요었습니다요다])\n'
    parts = re.split(pattern, text)
    return [p.strip() for p in parts if len(p.strip()) >= min_len]

def map_terms_to_sentences(terms: list[str], sentences: list[str]) -> dict[str, list[str]]:
    mapped = {}
    for term in terms:
        hits = [s for s in sentences if term in s]
        if hits:
            mapped[term] = list(dict.fromkeys(hits))
    return mapped

def combine_pdfs_to_one_csv(pdf_dir: str,
                            terms: list[str],
                            output_csv: str,
                            min_len: int = 10):
    # 1) pdf 폴더에서 .pdf 파일들 자동 수집
    pdf_files = [
        os.path.join(pdf_dir, fn)
        for fn in os.listdir(pdf_dir)
        if fn.lower().endswith(".pdf")
    ]

    combined_rows = []
    # 2) PDF별 처리
    for pdf_path in pdf_files:
        base = os.path.splitext(os.path.basename(pdf_path))[0]
        raw = extract_text_from_pdf(pdf_path)
        sentences = split_sentences(raw, min_len=min_len)
        mapped = map_terms_to_sentences(terms, sentences)

        for term, examples in mapped.items():
            for ex in examples:
                combined_rows.append({
                    'pdf': base,
                    'term': term,
                    'example': ex
                })
        print(f" • {base}.pdf → {len(mapped)}개 용어 매핑")


    df_all = pd.DataFrame(combined_rows)

    # 3-1) “term + example” 쌍 단위로 중복 제거
    df_all = df_all.drop_duplicates(subset=['term', 'example'])

    df_all = df_all.sort_values(by='term')

    # 4) 용례 개수 확인
    total_examples = len(df_all)
    print(f"\n▶ 총 용례 개수: {total_examples}건\n")

    print("▶ 용어별 용례 개수:")
    print(df_all['term'].value_counts(), "\n")

    print("▶ PDF별 용례 개수:")
    print(df_all['pdf'].value_counts(), "\n")

    # 5) CSV로 저장
    df_all.to_csv(output_csv, index=False, encoding="utf-8-sig")
    print(f"✔ 최종 결과가 '{output_csv}' 에 저장되었습니다.")

In [5]:
if __name__ == "__main__":
    weapon_df = pd.read_csv(
          '/content/drive/MyDrive/Hanhwa/crawling_project/crawling_project/data/무기의세계/무기의세계_불필요한것 제외.csv',
        encoding="utf-8-sig"
    )
    weapon_terms = weapon_df['용어'].tolist()

    pdf_folder =  "/content/drive/MyDrive/Hanhwa/crawling_project/crawling_project/data/"
    output_file ="/content/drive/MyDrive/Hanhwa/crawling_project/crawling_project/data/test_resid.csv"

    combine_pdfs_to_one_csv(pdf_folder, weapon_terms, output_file)

 • 국방과학기술플러스 250호.pdf → 17개 용어 매핑
 • 국방과학기술플러스 249호.pdf → 16개 용어 매핑
 • 국방과학기술플러스_248호(20년도+2분기).pdf → 17개 용어 매핑
 • 국방과학기술플러스_246호(19년도+4분기).pdf → 4개 용어 매핑
 • 국방과학기술플러스_247호(20년도+1분기).pdf → 8개 용어 매핑
 • 국방과학기술플러스_244호(2019년+2분기).pdf → 4개 용어 매핑
 • 국방과학기술플러스_245호(2019년+3분기).pdf → 0개 용어 매핑
 • 국방과학기술플러스_242호(2018년+4분기).pdf → 17개 용어 매핑
 • 국방과학기술플러스_243호(2019년+1분기).pdf → 0개 용어 매핑
 • 국방과학기술플러스_239호(2018년+1분기).pdf → 5개 용어 매핑
 • 국방과학기술플러스_238호(2017년+4분기).pdf → 2개 용어 매핑
 • 국방과학기술플러스_237호(2017년+3분기).pdf → 4개 용어 매핑
 • 국방과학기술플러스_236호(2017년+2분기).pdf → 2개 용어 매핑
 • 국방과학기술플러스_235호(2017년+1분기).pdf → 14개 용어 매핑
 • 국방과학기술플러스_241호(2018년+3분기).pdf → 1개 용어 매핑
 • 국방과학기술플러스_240호(2018년+2분기).pdf → 1개 용어 매핑

▶ 

# 최종 함수(전처리 과정)

In [6]:
import re
import pandas as pd

# --- 1) CSV 로드 ---
weapon_ex = pd.read_csv(
    '/content/drive/MyDrive/Hanhwa/crawling_project/crawling_project/data/test_resid.csv',
    encoding='utf-8-sig'
)

# --- 2) all_examples 구성 (pdf + example 튜플) ---
all_examples = []
for _, row in weapon_ex.iterrows():
    pdf = row['pdf']
    exs = row['example']
    if isinstance(exs, str):
        try:
            exs = eval(exs)
        except:
            exs = [exs]
    for ex in exs:
        all_examples.append((pdf, ex))

# --- 3) 필터링 함수 (튜플 버전) ---
def filter_examples(terms, examples, max_run=20, max_space_run=5, max_len=100):
    matched = {}
    for term in terms:
        pat = re.compile(r'(?<!\d)' + re.escape(term) + r'(?!\d)')
        hits = []
        for pdf, ex in examples:
            # 1) 용어 매칭
            if not pat.search(ex):
                continue
            # 2) 이중 줄바꿈 제거
            if "\n\n" in ex:
                continue
            # 3) 길이 제한
            if len(ex) > max_len:
                continue
            # 4) 긴 무공백(run) 검사
            runs = re.findall(r'[가-힣A-Za-z0-9]+', ex)
            if any(len(r) > max_run for r in runs):
                continue
            # 5) 연속 공백(run) 검사
            if ' ' * max_space_run in ex:
                continue
            # 6) “다.” 로 끝나지 않는 문장 제거
            if not ex.strip().endswith("다."):
                continue
            # 모두 통과하면, (pdf, example) 기록
            hits.append((pdf, ex))
        # 순서 유지하며 중복 제거
        matched[term] = list(dict.fromkeys(hits))
    return matched

# --- 4) terms 리스트 ---
terms = weapon_ex['term'].unique().tolist()

# --- 5) 필터링 수행 ---
filtered_dict = filter_examples(terms, all_examples,
                                max_run=20,
                                max_space_run=3,
                                max_len=200)

# --- 6) CSV 저장용 DataFrame으로 변환 ---
rows = []
for term, recs in filtered_dict.items():
    for pdf, ex in recs:
        rows.append({
            'term': term,
            'pdf': pdf,
            'example': ex
        })

df_out = pd.DataFrame(rows)

# term 기준 정렬
df_out = df_out.sort_values('term')


# --- 7) CSV로 저장 ---
output_path = "/content/drive/MyDrive/Hanhwa/crawling_project/crawling_project/data/국방과학연구소_나머지_용례매핑.csv"
df_out.to_csv(output_path, index=False, encoding="utf-8-sig")
print(f"✔ 저장 완료: {output_path}")

✔ 저장 완료: /content/drive/MyDrive/Hanhwa/crawling_project/crawling_project/data/국방과학연구소_나머지_용례매핑.csv


In [8]:
have_example_term = [term for term, exs in filtered_dict.items() if exs]
print(have_example_term) # 용례가 있는 용어
num_terms_with_examples = sum(1 for exs in filtered_dict.values() if exs)
num_examples_with_examples = sum(len(exs) for exs in filtered_dict.values() if exs)
print(f"용례가 있는 용어의 개수: {num_terms_with_examples}개") # 43개(50 -> 43)
print(f"총 용례의 개수: {num_examples_with_examples}개") # 334개(562 -> 334)

['5세대 전투기', 'A-10', 'IED', 'J-20', 'P-3C', 'RPG-7', 'SM-3', 'Su-30', 'UH-60 블랙호크', 'X-37B', '공격기', '공대공 미사일', '공중급유기', '근접전투', '기뢰', '기총', '다연장로켓포', '대륙간탄도미사일', '독도함', '라팔', '무인전투기', '미사일 방어체계', '박격포', '벙커버스터', '사드', '세종대왕함', '수리온', '신궁', '아파치', '어뢰', '연안전투함', '장갑차', '장사정포', '전익기', '전자전기', '정찰기', '정찰위성', '지뢰', '천궁', '킬체인', '하푼', '함포', '현궁']
용례가 있는 용어의 개수: 43개
총 용례의 개수: 334개


In [9]:
# 결과 확인
for term, exs in filtered_dict.items():
    if exs:
        print(f"{term} ({len(exs)}건):")
        for ex in exs:
            print("  -", ex)
        print()

5세대 전투기 (3건):
  - ('국방과학기술플러스 250호', '전 세계에서 저피탐 기술이 가장 발달된 것으로 알려진 미국은 최근 저피탐 성능을 키워드로 하는 5세대 전투기 개발에 이어 6세대 전투기 개발계획을 발표했으며 중국, 일본 등 한반도를 둘러싼 주변 강대국들도 속속들이 자국의 저피탐 무기체계 개발 계획 및 결과를 발표하고 있다.')
  - ('국방과학기술플러스_242호(2018년+4분기)', '5세대 전투기인 F-22, F-35 개발 이후 스텔스 성능은 군사용 항공기에 필수적인 적용기술이 되었으며 스텔스 항공기는 적 레이더에 탐지되는 레이더 단면적(RCS)을 최소로 하는 것이 목표이다.')
  - ('국방과학기술플러스 249호', '러시아의 5세대 전투기인 Su-57(일명 PAK-PA)에는 Tikhomirov NIIP 연구소에서 개발한 2종류의 주파수 대역을 가진 레이다가 장착되어 있다.')

A-10 (1건):
  - ('국방과학기술플러스_242호(2018년+4분기)', 'HA-10이라는 무인기의 개발도 병행하였는데 7~15km 상공을 24시간 체공 비행하는것을 목표로 하였다.')

IED (2건):
  - ('국방과학기술플러스_239호(2018년+1분기)', 'IED에 의한 피해를 줄이기 위해서 재머를 켠 상태에서 이동을 하지만, 이 상황에서 통신장비와 간섭문제가 발생하여 원활한 통신에 문제가 발생하는 사례가 잦았다.')
  - ('국방과학기술플러스_239호(2018년+1분기)', '그 중에서 가장 대표적인 내용으로써, 급조폭발물에 의한 공격을 방지하고자 개발되고 배치된 Counter-IED (Counter Improvised Explosive Device)와 통신장비와의 간섭문제가 있다.')

J-20 (1건):
  - ('국방과학기ᄉ

In [7]:
df_out

,term,pdf,example
0,5세대 전투기,국방과학기술플러스 250호,전 세계에서 저피탐 기술이 가장 발달된 것으로 알려진 미국은 최근 저피탐 성능을 키...
1,5세대 전투기,국방과학기술플러스_242호(2018년+4분기),"5세대 전투기인 F-22, F-35 개발 이후 스텔스 성능은 군사용 항공기에 필수적..."
2,5세대 전투기,국방과학기술플러스 249호,러시아의 5세대 전투기인 Su-57(일명 PAK-PA)에는 Tikhomirov NI...
3,A-10,국방과학기술플러스_242호(2018년+4분기),HA-10이라는 무인기의 개발도 병행하였는데 7~15km 상공을 24시간 체공 비행...
4,IED,국방과학기술플러스_239호(2018년+1분기),"IED에 의한 피해를 줄이기 위해서 재머를 켠 상태에서 이동을 하지만, 이 상황에서..."
...,...,...,...
332,현궁,국방과학기술플러스 249호,"현궁의 경우, 무기체계가 소형이었기 때문에 하나의 열전지로 모든 전원시스템을 감당했..."
329,현궁,국방과학기술플러스 249호,순수한 국내 열전지만의 적용은 보병용 중거리 유도무기인 현궁에서부터 시작되었다.
330,현궁,국방과학기술플러스_248호(20년도+2분기),이는 발사 후 망각 유도(Fire & Forget) 방식으로 보병용 중거리 유도무기...
331,현궁,국방과학기술플러스_248호(20년도+2분기),75인치 유도로켓 비궁과 보병용 중거리유도무기 현궁에 적용된 탐색기 추적알고리듬 기...


# csv 저장

In [10]:
def xlsx_to_csv(input_xlsx: str, output_csv: str, sheet_name=0):

    # 1) 엑셀 파일 로드
    df = pd.read_excel(input_xlsx, sheet_name=sheet_name, engine='openpyxl')

    # 2) CSV로 저장 (UTF-8 BOM 포함)
    df.to_csv(output_csv, index=False, encoding='utf-8-sig')
    print(f" '{input_xlsx}' → '{output_csv}' 변환 완료 (encoding='utf-8-sig')")

In [11]:
if __name__ == "__main__":
    input_xlsx = '/content/drive/MyDrive/Hanhwa/crawling_project/crawling_project/data/국방과학연구소_나머지_용례매핑.xlsx'
    output_csv = '/content/drive/MyDrive/Hanhwa/crawling_project/crawling_project/data/국방과학연구소_무기의세계_용례매핑_나머지_전처리후.csv'     # 저장할 csv 파일 경로
    xlsx_to_csv(input_xlsx, output_csv)

 '/content/drive/MyDrive/Hanhwa/crawling_project/crawling_project/data/국방과학연구소_나머지_용례매핑.xlsx' → '/content/drive/MyDrive/Hanhwa/crawling_project/crawling_project/data/국방과학연구소_무기의세계_용례매핑_나머지_전처리후.csv' 변환 완료 (encoding='utf-8-sig')


# 용어별 리스트로 csv 저장

In [12]:
def group_manual_csv(input_csv: str, output_csv: str):
    # 1) 기존 CSV 로드 (UTF-8 BOM 포함)
    df = pd.read_csv(input_csv, encoding='utf-8-sig')

    # 2) 컬럼 이름 확인 (필요시 조정)
    print("원본 컬럼:", df.columns.tolist())

    # 3) term별로 pdf와 example을 리스트로 묶기
    grouped = (
        df
        .groupby('term')
        .agg({
            'pdf': lambda s: list(dict.fromkeys(s)),   # PDF 출처 중복 제거 후 리스트
            'example': list                            # 예시 문장 리스트
        })
        .reset_index()
        .rename(columns={
            'pdf': 'sources',
            'example': 'examples'
        })
    )

    # 4) 결과 확인
    print(grouped.head())

    # 5) CSV로 저장 (UTF-8 BOM 포함)
    grouped.to_csv(output_csv, index=False, encoding='utf-8-sig')
    print(f"✔ '{output_csv}'에 저장 완료!")

if __name__ == "__main__":
    input_csv = '/content/drive/MyDrive/Hanhwa/crawling_project/crawling_project/data/국방과학연구소_무기의세계_용례매핑_나머지_전처리후.csv'
    output_csv = "/content/drive/MyDrive/Hanhwa/crawling_project/crawling_project/data/국방과학연구소_무기의세계_용례매핑_나머지_전처리후_일대다.csv"
    group_manual_csv(input_csv, output_csv)

원본 컬럼: ['term', 'pdf', 'example']
      term                                            sources  \
0  5세대 전투기  [국방과학기술플러스 250호, 국방과학기ᄉ...   
1      IED    [국방과학기술플러스_239호(2018년+1분기)]   
2     J-20                    [국방과학기술플러스 249호]   
3     P-3C    [국방과학기술플러스_247호(20년도+1분기)]   
4    RPG-7    [국방과학기술플러스_235호(2017년+1분기)]   

                                            examples  
0  [전 세계에서 저피탐 기술이 가장 발달된 것으로 알려진 미국은 최근 저피탐 성능을 ...  
1  [IED에 의한 피해를 줄이기 위해서 재머를 켠 상태에서 이동을 하지만, 이 상황에...  
2  [3세대 AESA 레이다 기술은 J-20 전투기 포함 다수의 전투기 탑재를 목표로 ...  
3  [1990년대에는 Helium원자의 Zeeman 효과를 적용한 자기장 센서가 개발되...  
4  [최근 북한도 이라크전 등 현대전의 전쟁교훈을 분석하여 시가지 전투에 적합한 전술과...  
✔ '/content/drive/MyDrive/Hanhwa/crawling_project/crawling_project/data/국방과학연구소_무기의세계_용례매핑_나머지_전처리후_일대다.csv'에 저장 완료!
